# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [106]:
import pandas as pd
import requests
pd.set_option('mode.chained_assignment', None)

In [107]:
# 1) seu código aqui
df = pd.read_csv('SINASC_RO_2019.csv')
print(df.shape)
df.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [108]:
# 2) seu código aqui
df.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [109]:
# 3) seu código aqui
df_nova = df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df_nova

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [110]:
# 4) seu código aqui
df_nova.dropna(subset=['APGAR5'], inplace=True)
df_nova.APGAR5.isnull().sum()

0

In [111]:
# 5) seu código aqui
df_nova['CONSULTAS'].fillna(9, inplace=True)
df_nova.CONSULTAS.isnull().sum()

0

In [112]:
df_nova['ESTCIVMAE'].fillna(9, inplace=True)
df_nova.ESTCIVMAE.isnull().sum()

0

In [113]:
# 6) Seu código aqui
df_nova['QTDFILVIVO'].fillna(0, inplace=True)
df_nova.QTDFILVIVO.isnull().sum()

0

In [114]:
# 7) seu código aqui
df_nova.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

ARRUMANDO COLUNA ESCMAE

In [115]:
df_nova.ESCMAE.value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

In [116]:
df_nova['ESCMAE'] = df_nova.groupby('APGAR5')['ESCMAE'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [117]:
df_nova.ESCMAE.isnull().sum()

0

ARRUMANDO COLUNA GESTACAO

In [118]:
df_nova.GESTACAO.value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [119]:
df_nova.groupby('APGAR5')['GESTACAO'].apply(lambda x: x.mode().iloc[0])

APGAR5
0.0     22 a 27 semanas
1.0     37 a 41 semanas
2.0     22 a 27 semanas
3.0     22 a 27 semanas
4.0     32 a 36 semanas
5.0     37 a 41 semanas
6.0     37 a 41 semanas
7.0     37 a 41 semanas
8.0     37 a 41 semanas
9.0     37 a 41 semanas
10.0    37 a 41 semanas
Name: GESTACAO, dtype: object

In [120]:
df_nova['GESTACAO'] = df_nova.groupby('APGAR5')['GESTACAO'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [121]:
df_nova.GESTACAO.isnull().sum()

0

ARRUMANDO COLUNA GRAVIDEZ

In [122]:
df_nova.GRAVIDEZ.value_counts()

Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [123]:
df_nova['GRAVIDEZ'].fillna('nenhuma', inplace=True)
df_nova.GRAVIDEZ.isnull().sum()

0

In [124]:
df_nova.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [125]:
# 8) seu código aqui
df_nova['APGAR5_CATEGORIA'] = pd.cut(df_nova['APGAR5'], bins=[0, 3, 5, 7, 10], labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'])
df_nova['APGAR5_CATEGORIA'].value_counts(normalize=True)*100

normal              98.346217
asfixia leve         1.189237
asfixia moderada     0.252713
asfixia severa       0.211833
Name: APGAR5_CATEGORIA, dtype: float64

In [126]:
# 9) seu código aqui
tamanho = len(df_nova.columns)

novos_nomes = ['local_de_nascimento', 'idade_mae', 'estado_civil', 'escolaridade_mae', 'quantidade_filhos', 'semana_gestacao', 'numero_gestacoes', 'quantidade_consultas', 'nivel_asfixia', 'faixa_asfixia']

for i in range(tamanho):
    df_nova.rename(columns={df_nova.columns[i]: novos_nomes[i]}, inplace=True)


In [127]:
df_nova

,local_de_nascimento,idade_mae,estado_civil,escolaridade_mae,quantidade_filhos,semana_gestacao,numero_gestacoes,quantidade_consultas,nivel_asfixia,faixa_asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [128]:
df_nova.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26925 entries, 0 to 27027
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   local_de_nascimento   26925 non-null  int64   
 1   idade_mae             26925 non-null  int64   
 2   estado_civil          26925 non-null  float64 
 3   escolaridade_mae      26925 non-null  object  
 4   quantidade_filhos     26925 non-null  float64 
 5   semana_gestacao       26925 non-null  object  
 6   numero_gestacoes      26925 non-null  object  
 7   quantidade_consultas  26925 non-null  int64   
 8   nivel_asfixia         26925 non-null  float64 
 9   faixa_asfixia         26908 non-null  category
dtypes: category(1), float64(3), int64(3), object(3)
memory usage: 2.1+ MB
